# Setup

In [1]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import StoppingCriteria, StoppingCriteriaList
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import time
import json

nltk.download('punkt_tab')

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/praveen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Qwen2.5-Coder-14B-bnb-4bit",
    model_name = "outputs/checkpoint-500",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.628 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.2.15 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


# Preparing dataset

In [3]:
repo_name = 'huly_platform'

def format_train_example(example):
    filePath = example['filePath']
    prefix = example['prefix']
    suffix = example['suffix']
    middle = example['middle']
    return { 'text': f'<|repo_name|>{repo_name}\n<|file_sep|>{filePath}\n<|fim_prefix|>{prefix}<|fim_suffix|>{suffix}<|fim_middle|>{middle}<|endoftext|>' }

def format_test_example(example):
    filePath = example['filePath']
    prefix = example['prefix']
    suffix = example['suffix']
    return { 'text': f'<|repo_name|>{repo_name}\n<|file_sep|>{filePath}\n<|fim_prefix|>{prefix}<|fim_suffix|>{suffix}<|fim_middle|>' }

In [4]:
dataset = load_dataset('json', data_files={ 'train': 'dataset.train.jsonl', 'test': 'dataset.test.jsonl' }) 
train_dataset = dataset['train'].map(format_train_example)
test_dataset = dataset['test'].map(format_test_example)

print(f'Num train samples: {train_dataset.num_rows}')
print(f'Num test samples: {test_dataset.num_rows}')

Num train samples: 43733
Num test samples: 1149


In [ ]:
print(train_dataset[3140]['text'])

# Evaluate pre-trained model

In [5]:
stop_words = ['<|endoftext|>', '<|file_sep|>']
stop_ids = tokenizer.convert_tokens_to_ids(stop_words)
max_new_tokens = 1280

class StopTokenCriteria(StoppingCriteria):
    def __init__(self, stop_ids):
        self.stop_ids = set(stop_ids)

    def __call__(self, input_ids, scores, **kwargs):
        return torch.isin(input_ids[:, -1], torch.tensor(list(self.stop_ids), device=input_ids.device))

stopping_criteria = StoppingCriteriaList([StopTokenCriteria(stop_ids)])

def generate_completions(input_ids, attention_mask):
    with torch.no_grad():
        start_ts = time.time()
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, stopping_criteria=stopping_criteria)
        elapsed = time.time() - start_ts
        num_input_tokens = input_ids.shape[1]
        num_generated_tokens = output.shape[1] - num_input_tokens
        print(f'Generated {num_generated_tokens} tokens in {elapsed:.2f} seconds ({num_generated_tokens/elapsed:.2f} tokens/s)')
    output = output[:, input_ids.shape[1]:-1]
    generated_texts = tokenizer.batch_decode(output, skip_special_tokens=True)
    return generated_texts

In [6]:
batch_size = 4
# generated_file = 'generated.test.jsonl'
generated_file = 'generated-post-finetune.test.jsonl'

def collate_fn(batch):
    texts = [example['text'] for example in batch]
    tokenized = tokenizer(texts, padding=True, truncation=True, max_length=max_seq_length - max_new_tokens, return_tensors="pt")
    tokenized['middle'] = [example['middle'] for example in batch]
    tokenized['text'] = [example['text'] for example in batch] 
    tokenized['prefix'] = [example['prefix'] for example in batch]
    tokenized['suffix'] = [example['suffix'] for example in batch]
    return tokenized
data_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

FastLanguageModel.for_inference(model)
device = 'cuda'

correct = 0
total = 0
bleu_scores = []

def compute_bleu_score(generated_texts, expected_texts):
    expected = nltk.word_tokenize(expected_texts)
    generated = nltk.word_tokenize(generated_texts)
    return sentence_bleu([expected], generated, smoothing_function=SmoothingFunction().method1)

with open(generated_file, 'w') as f:
    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        print(f'input shape:', input_ids.shape)
        attention_mask = batch['attention_mask'].to(device)

        start_ts = time.time()
        generated_texts = generate_completions(input_ids, attention_mask)
        elapsed = time.time() - start_ts
        print(f'Generated {len(generated_texts)} completions in {elapsed:.2f} seconds')

        for generated, expected, input_prompt, prefix, suffix in zip(generated_texts, batch['middle'], batch['text'], batch['prefix'], batch['suffix']):
            if generated == expected:
                correct += 1
            total += 1
            bleu_score = compute_bleu_score(generated, expected)
            bleu_scores.append(bleu_score)

            f.write(json.dumps({
                'text': input_prompt,
                'generated': generated,
                'expected': expected,
                'prefix': prefix,
                'suffix': suffix,
                'bleu_score': bleu_score,
            }) + '\n')

            if total % 2 == 0:
                print(f'Generated: {generated}')
                print('-' * 40)
                print(f'Expected: {expected}')
                print('=' * 80)
                print(f'BLEU score: {bleu_score}')

print(f'Correct: {correct}')
print(f'Total: {total}')
print(f'Accuracy: {correct / total}')
print(f'Average BLEU score: {sum(bleu_scores) / len(bleu_scores)}')

  0%|          | 0/288 [00:00<?, ?it/s]

input shape: torch.Size([4, 1559])


  0%|          | 1/288 [00:08<42:21,  8.85s/it]

Generated 51 tokens in 8.80 seconds (5.80 tokens/s)
Generated 4 completions in 8.80 seconds
Generated: <ChannelMembers
      {object}
      {disabledRemoveFor}
      on:click={openSelectUsersPopup}
      on:remove={removeMember}
      {readonly}
    />
----------------------------------------
Expected: <ChannelMembers
      ids={Array.from(members)}
      disableRemoveFor={disabledRemoveFor}
      {readonly}
      on:add={openSelectUsersPopup}
      on:remove={removeMember}
    />
BLEU score: 0.47630546948745894
Generated: <div>
                {#each data.assignees as assignee, i}
                  {#if i},
                  {/if}
                  <a href={assignee.url}>{assignee.login}</a>
                {/each}
              </div>
----------------------------------------
Expected: <div>
                {#each data.assignees as assignee}
                  <a href={assignee.url}>@{assignee.login}</a>
                {/each}
              </div>
BLEU score: 0.6306012316200106
input 

  1%|          | 2/288 [00:20<49:15, 10.33s/it]

...

Generated 49 tokens in 7.42 seconds (6.60 tokens/s)
Generated 4 completions in 7.42 seconds
Generated: {
    if (selected !== undefined && dragswap(ev, i, cat._id)) {
      elements[i].scrollIntoView({ behavior: 'instant', block: 'nearest' })
      moveState(i)
    }
  }
----------------------------------------
Expected: {
    if (readonly) return
    const s = selected as number

    if (dragswap(ev, i, targetCategory)) {
      ;[states[i], states[s]] = [states[s], states[i]]
      selected = i
    }
  }
BLEU score: 0.1648415502442912
Generated: {
      await updateIssueRelation(client, refDocument, value, 'blockedBy', operation)
    }
----------------------------------------
Expected: {
      if (type !== 'blockedBy') {
        await updateIssueRelation(client, refDocument as Issue, value, prop, operation)
      }

      return
    }
BLEU score: 0.30587838334797846
input shape: torch.Size([4, 1986])


100%|█████████▉| 287/288 [1:07:38<00:07,  7.42s/it]

Generated 29 tokens in 6.59 seconds (4.40 tokens/s)
Generated 4 completions in 6.59 seconds
Generated: selectedDoneStates.size === 0 && !withoutDone
----------------------------------------
Expected: selectedDS.length === 0
BLEU score: 0.06985342056580097
Generated: <div class="hulyHeader-buttonsGroup presence no-print">
        <slot name="presence" {doubleRow} />
      </div>
----------------------------------------
Expected: <div class="hulyHeader-buttonsGroup presence no-print">
        <slot name="presence" {doubleRow} />
      </div>
BLEU score: 1.0
input shape: torch.Size([1, 811])


100%|██████████| 288/288 [1:07:39<00:00, 14.10s/it]

Generated 30 tokens in 0.96 seconds (31.15 tokens/s)
Generated 1 completions in 0.96 seconds
Correct: 416
Total: 1149
Accuracy: 0.36205395996518713
Average BLEU score: 0.6060399681578575


# Run SFT

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        run_name='ctrl-space-qwen-2.5-14b-sft',
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        save_steps=500,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth 2025.2.15 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 43,733 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 5,466
 "-____-"     Number of trainable parameters = 68,812,800
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: prvnsmpth (casepro) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,0.982000
2,1.093700
3,0.923200
4,1.036400
5,0.705700
6,0.888000
7,0.898100
8,0.922700
9,0.608000
10,0.980200


# Inference

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
example = dataset[1813]
print(example['filePath'])
print(example['prefix'])
print('\n==================\n')
print(example['middle'])
print('\n==================\n')
print(example['suffix'])

In [ ]:
example_str = format_example(example)['text'].split('<|fim_middle|>')[0] + '<|fim_middle|>'
inputs = tokenizer(example_str, return_tensors="pt").to("cuda")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=1024, use_cache = True, do_sample=True, temperature=0.8, top_p=0.95)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text[0].split("<|fim_middle|>")[-1])

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=1024, use_cache = True, do_sample=True, temperature=0.8, top_p=0.95)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text[0].split("<|fim_middle|>")[-1])

In [ ]:
all_special_tokens = tokenizer.all_special_tokens
all_special_ids = tokenizer.all_special_ids

print("Special Tokens and their IDs:")
for token, token_id in zip(all_special_tokens, all_special_ids):
    print(f"Token: {token}, ID: {token_id}")

# Save model

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")